In [1]:
import h5py  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

# Preprocessing

In [2]:
#If using GPU.
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
#read database.
f = h5py.File('GWdatabase.h5','r')   
f.keys()   

<KeysViewHDF5 ['reduced_data', 'waveforms', 'yeofrho']>

In [4]:
#create a list that contains all the failure cases.
fail_num = []
index = 0
for item in f['reduced_data']['tbounce(s)']:
    if(item == -1):
        fail_num.append(index)
    index += 1
fail_case = []
for index in fail_num:
    fail_case.append([f['reduced_data']['A(km)'][index],f['reduced_data']['omega_0(rad|s)'][index],f['reduced_data']['EOS'][index]])
fail_list= []
for item in fail_case:
    tmp = str(item[2]).split("b'")[1].split("'")[0]
    tmp = "A" + str(int(item[0])) + "w" + str(item[1]) + "0_" + tmp
    fail_list.append(tmp)    

In [5]:
#A_num is the list of all the cases that have the particular A value
A_num = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        #if(float(item.split('_')[0].split('w')[0].split('A')[1]) == 467):
         if(str(item.split('_')[1]) != 'BHBL'):
            A_num.append(index)
    index+=1
len(A_num)

1667

# Choose which label we want to predict 

In [6]:
#get the label of EOS.
y_train = []
y_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            y_train.append(str(item.split('_')[1]))
        else:
            y_test.append(str(item.split('_')[1]))
    index += 1

In [7]:
y_test = np.where(y_test == "BHBL", "B", y_test)
y_train = np.where(y_train == "BHBLP", "B", y_train)
y_train = np.where(y_train == "GShenFSU1.7", "G", y_train)
y_train = np.where(y_train == "GShenFSU2.1", "G", y_train)
y_train = np.where(y_train == "GShenNL3", "G", y_train)
y_train = np.where(y_train == "HSDD2", "B", y_train)
y_train = np.where(y_train == "HSFSG", "B", y_train)
y_train = np.where(y_train == "HSIUF", "B", y_train)
y_train = np.where(y_train == "HSNL3", "B", y_train)
y_train = np.where(y_train == "HSTM1", "B", y_train)
y_train = np.where(y_train == "HSTMA", "B", y_train)
y_train = np.where(y_train == "HShen", "H", y_train)
y_train = np.where(y_train == "HShenH", "H", y_train)
y_train = np.where(y_train == "LS180", "L", y_train)
y_train = np.where(y_train == "LS220", "L", y_train)
y_train = np.where(y_train == "LS375", "L", y_train)
y_train = np.where(y_train == "SFHo", "B", y_train)
y_train = np.where(y_train == "SFHx", "B", y_train)

In [8]:
y_test = np.where(y_test == "BHBL", "B", y_test)
y_train = np.where(y_train == "BHBLP", "B", y_train)
y_train = np.where(y_train == "GShenFSU1.7", "G", y_train)
y_train = np.where(y_train == "GShenFSU2.1", "G", y_train)
y_train = np.where(y_train == "GShenNL3", "G", y_train)
y_train = np.where(y_train == "HSDD2", "B", y_train)
y_train = np.where(y_train == "HSFSG", "B", y_train)
y_train = np.where(y_train == "HSIUF", "B", y_train)
y_train = np.where(y_train == "HSNL3", "B", y_train)
y_train = np.where(y_train == "HSTM1", "B", y_train)
y_train = np.where(y_train == "HSTMA", "B", y_train)
y_train = np.where(y_train == "HShen", "H", y_train)
y_train = np.where(y_train == "HShenH", "H", y_train)
y_train = np.where(y_train == "LS180", "L", y_train)
y_train = np.where(y_train == "LS220", "L", y_train)
y_train = np.where(y_train == "LS375", "L", y_train)
y_train = np.where(y_train == "SFHo", "B", y_train)
y_train = np.where(y_train == "SFHx", "B", y_train)

# Get data 

In [9]:
#read image data.
from PIL import Image
import numpy as np
X_train = []
X_test = []
index = 0
for item in f['waveforms']:
    if(item not in fail_list):
        if(index in A_num):
            title = 'Final_EOS/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_train.append(np.array(image))
        else:
            title = 'Final_EOS/'+ str(index) + '.jpeg'
            image = Image.open(title).convert('L')
            X_test.append(np.array(image))
    index += 1

In [10]:
#transform the data type to numpy array
X_train = np.array(X_train)
X_test = np.array(X_test)

# Model

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import backend as K
import os
import gc

In [12]:
X_train.shape

(1667, 256, 256)

In [13]:
#transfer the data shape for model training
X_train = X_train.reshape(X_train.shape[0], 256, 256, 1)
X_test = X_test.reshape(X_test.shape[0], 256, 256, 1)

In [14]:
#using one hot to encode the label.
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)

In [15]:
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from keras.layers import Concatenate
from keras.models import Model

In [16]:
#our model
model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(256, 256, 1), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (5, 5), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))
optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [17]:
history = model.fit(x=X_train, y=y_train, epochs=100, batch_size=16)

Epoch 1/100
105/105 [==============================] - 9s 60ms/step - loss: 2.8292 - acc: 0.3942
Epoch 2/100
105/105 [==============================] - 6s 54ms/step - loss: 1.2919 - acc: 0.4957
Epoch 3/100
105/105 [==============================] - 6s 54ms/step - loss: 1.2081 - acc: 0.5605
Epoch 4/100
105/105 [==============================] - 6s 55ms/step - loss: 1.2137 - acc: 0.5430
Epoch 5/100
105/105 [==============================] - 6s 54ms/step - loss: 1.1881 - acc: 0.5515
Epoch 6/100
105/105 [==============================] - 6s 54ms/step - loss: 1.2090 - acc: 0.5365
Epoch 7/100
105/105 [==============================] - 6s 54ms/step - loss: 1.1786 - acc: 0.5615
Epoch 8/100
105/105 [==============================] - 6s 54ms/step - loss: 1.1572 - acc: 0.5716
Epoch 9/100
105/105 [==============================] - 6s 52ms/step - loss: 1.1859 - acc: 0.5547
Epoch 10/100
105/105 [==============================] - 6s 52ms/step - loss: 1.1586 - acc: 0.5627
Epoch 11/100
105/105 [=======

105/105 [==============================] - 6s 55ms/step - loss: 0.0173 - acc: 0.9942
Epoch 85/100
105/105 [==============================] - 6s 54ms/step - loss: 0.0151 - acc: 0.9955
Epoch 86/100
105/105 [==============================] - 6s 55ms/step - loss: 0.0090 - acc: 0.9977
Epoch 87/100
105/105 [==============================] - 6s 55ms/step - loss: 0.0064 - acc: 0.9982
Epoch 88/100
105/105 [==============================] - 6s 55ms/step - loss: 0.0060 - acc: 0.9992
Epoch 89/100
105/105 [==============================] - 6s 54ms/step - loss: 0.0040 - acc: 0.9995
Epoch 90/100
105/105 [==============================] - 6s 55ms/step - loss: 0.0067 - acc: 0.9990
Epoch 91/100
105/105 [==============================] - 6s 55ms/step - loss: 0.0382 - acc: 0.9930
Epoch 92/100
105/105 [==============================] - 6s 55ms/step - loss: 0.0043 - acc: 0.9999
Epoch 93/100
105/105 [==============================] - 6s 55ms/step - loss: 0.0086 - acc: 0.9986
Epoch 94/100
 12/105 [==>........

KeyboardInterrupt: 

In [24]:
#if the GPU is out of memory while testing, we may need to save the trained model, restart, then load the model. 
model.save('Multitest_BHBL.h5')

In [17]:
model = keras.models.load_model('Multitest_BHBL.h5')

In [18]:
#check the accuracy.
answer = np.argmax(model.predict(X_test), axis=1)
answer_onehot = np.zeros((answer.size, answer.max()+1))
answer_onehot[np.arange(answer.size),answer] = 1
answer_decode = label_decode(label_encoder, answer_onehot)
y_test_tmp = y_test
index = 0
correct = 0
for item in answer_decode:
    if(item == y_test_tmp[index]):
        correct += 1
    else:
        print(index)
    index += 1
correct/index

1.0